In [ ]:
# Install required libraries
!pip install datasets transformers evaluate

# Import libraries
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

# Load and preprocess dataset
dataset = load_dataset("mteb/tweet_sentiment_extraction")

# Use a fast tokenizer for efficiency
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Minimize the dataset size
small_train_dataset = tokenized_datasets["train"].shuffle(seed=25).select(range(300))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=25).select(range(300))

# Load a smaller model for faster training
model = GPT2ForSequenceClassification.from_pretrained("distilgpt2", num_labels=3)

# Define the accuracy metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Set training arguments
training_args = TrainingArguments(
    output_dir="test_trainer",
    num_train_epochs=1,  # Quick training with one epoch
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    evaluation_strategy="no",  # Skip evaluation during training to save time
    save_steps=500,  # Save checkpoint every 500 steps
    logging_dir="./logs",  # Directory for storing logs
    logging_steps=100
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()

# Evaluate the model
metrics = trainer.evaluate()
print("Evaluation Metrics:", metrics)

# Save the final model
trainer.save_model("final_model")
